<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Introduction to SQL

by [The Carpentries](https://training.galaxyproject.org/hall-of-fame/carpentries/), [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/), [Donny Vrins](https://training.galaxyproject.org/hall-of-fame/dirowa/), [Bazante Sanders](https://training.galaxyproject.org/hall-of-fame/bazante1/), [Avans Hogeschool](https://training.galaxyproject.org/hall-of-fame/avans-atgm/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- How can I get data from a database?
- How can I sort a query's results?
- How can I remove duplicate values from a query's results?
- How can I select subsets of data?
- How can I calculate new values on the fly?
- How do databases represent missing information?
- What special handling does missing information require?

**Objectives**

- Explain the difference between a table, a record, and a field.
- Explain the difference between a database and a database manager.
- Write a query to select all values for specific fields from a single table.
- Write queries that display results in a particular order.
- Write queries that eliminate duplicate values from data.
- Write queries that select records that satisfy user-specified conditions.
- Explain the order in which the clauses in a query are executed.
- Write queries that calculate new values for each selected record.
- Explain how databases represent missing information.
- Explain the three-valued logic databases use when manipulating missing information.
- Write queries that handle missing information correctly.

**Time Estimation: 3H**
</div>


<p>This tutorial will introduce you to Structured Query Language (SQL) which can be used to query databases!</p>
<blockquote class="comment" style="border: 2px solid #ffecc1; margin: 1em 0.2em">
<div class="box-title comment-title" id="comment"><i class="far fa-comment-dots" aria-hidden="true" ></i> Comment</div>
<p>This tutorial is <strong>significantly</strong> based on <a href="https://carpentries.org">the Carpentries</a> <a href="https://github.com/swcarpentry/sql-novice-survey/">Databases and SQL</a> lesson, which is licensed CC-BY 4.0.</p>
<p>Abigail Cabunoc and Sheldon McKay (eds): “Software Carpentry: Using Databases and SQL.”  Version 2017.08, August 2017,
<a href="https://github.com/swcarpentry/sql-novice-survey">github.com/swcarpentry/sql-novice-survey</a>, <a href="https://doi.org/10.5281/zenodo.838776">https://doi.org/10.5281/zenodo.838776</a></p>
<p>Adaptations have been made to make this work better in a GTN/Galaxy environment.</p>
</blockquote>
<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<div class="box-title agenda-title" id="agenda">Agenda</div>
<p>In this tutorial, we will cover:</p>
</blockquote>


In [1]:
# Install required packages # SQL alchemy
# pip install ipython-sql sqlalchemy

In [2]:
# Install SQL and SQLAlchemy libraries
%pip install ipython-sql sqlalchemy

# Download the SQLite database file
import urllib.request
url = "http://swcarpentry.github.io/sql-novice-survey/files/survey.db"
filename = "survey.db"
urllib.request.urlretrieve(url, filename)
print("Downloaded survey.db")


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Downloaded survey.db


In [3]:
# Import and configure SQL magic
import sqlalchemy
%load_ext sql
%sql sqlite:///survey.db
%config SqlMagic.displaycon = False


In [4]:
import prettytable
prettytable.DEFAULT = prettytable.PLAIN_COLUMNS  # or prettytable.DEFAULT_STYLE


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11232\3045698007.py:2: DeprecationWarning: the 'PLAIN_COLUMNS' constant is deprecated, use the 'TableStyle' enum instead
  prettytable.DEFAULT = prettytable.PLAIN_COLUMNS  # or prettytable.DEFAULT_STYLE


In [5]:
%%sql
SELECT * FROM person;


Done.


id,personal,family
dyer,William,Dyer
pb,Frank,Pabodie
lake,Anderson,Lake
roe,Valentina,Roerich
danforth,Frank,Danforth


In [6]:
# This code is not working
# This preamble sets up the sql "magic" for jupyter. Use %%sql in your cells to write sql!
# !python3 -m pip install ipython-sql sqlalchemy
# !wget -c http://swcarpentry.github.io/sql-novice-survey/files/survey.db
# import sqlalchemy
# engine = sqlalchemy.create_engine("sqlite:///survey.db")
# %load_ext sql
# %sql sqlite:///survey.db
# %config SqlMagic.displaycon=False

<h1 id="selecting-data">Selecting Data</h1>
<p>A relational database
is a way to store and manipulate information.
Databases are arranged as table.
Each table has columns (also known as fields) that describe the data,
and rows (also known as records) which contain the data.</p>
<p>When we are using a spreadsheet,
we put formulas into cells to calculate new values based on old ones.
When we are using a database,
we send commands
(usually called queries)
to a database manager:
a program that manipulates the database for us.
The database manager does whatever lookups and calculations the query specifies,
returning the results in a tabular form
that we can then use as a starting point for further queries.</p>
<p>Queries are written in a language called <abbr title="Structured Query Language">SQL</abbr>,
SQL provides hundreds of different ways to analyze and recombine data.
We will only look at a handful of queries,
but that handful accounts for most of what scientists do.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-changing-database-managers"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-changing-database-managers" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: Changing Database Managers</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>Many database managers — Oracle,
IBM DB2, PostgreSQL, MySQL, Microsoft Access, and SQLite —  understand
SQL but each stores data in a different way,
so a database created with one cannot be used directly by another.
However, every database manager
can import and export data in a variety of formats like .csv, SQL,
so it <em>is</em> possible to move information from one to another.</p>
</blockquote>
<p>Before we get into using <abbr title="Structured Query Language">SQL</abbr> to select the data, let’s take a look at the tables of the database we will use in our examples:</p>
<p><strong>Person</strong>: people who took readings.</p>
<table>
<thead>
<tr>
<th>id</th>
<th>personal</th>
<th>family</th>
</tr>
</thead>
<tbody>
<tr>
<td>dyer</td>
<td>William</td>
<td>Dyer</td>
</tr>
<tr>
<td>pb</td>
<td>Frank</td>
<td>Pabodie</td>
</tr>
<tr>
<td>lake</td>
<td>Anderson</td>
<td>Lake</td>
</tr>
<tr>
<td>roe</td>
<td>Valentina</td>
<td>Roerich</td>
</tr>
<tr>
<td>danforth</td>
<td>Frank</td>
<td>Danforth</td>
</tr>
</tbody>
</table>
<p><strong>Site</strong>: locations where readings were taken.</p>
<table>
<thead>
<tr>
<th>name</th>
<th>lat</th>
<th>long</th>
</tr>
</thead>
<tbody>
<tr>
<td>DR-1</td>
<td>-49.85</td>
<td>-128.57</td>
</tr>
<tr>
<td>DR-3</td>
<td>-47.15</td>
<td>-126.72</td>
</tr>
<tr>
<td>MSK-4</td>
<td>-48.87</td>
<td>-123.4</td>
</tr>
</tbody>
</table>
<p><strong>Visited</strong>: when readings were taken at specific sites.</p>
<table>
<thead>
<tr>
<th>id</th>
<th>site</th>
<th>dated</th>
</tr>
</thead>
<tbody>
<tr>
<td>619</td>
<td>DR-1</td>
<td>1927-02-08</td>
</tr>
<tr>
<td>622</td>
<td>DR-1</td>
<td>1927-02-10</td>
</tr>
<tr>
<td>734</td>
<td>DR-3</td>
<td>1930-01-07</td>
</tr>
<tr>
<td>735</td>
<td>DR-3</td>
<td>1930-01-12</td>
</tr>
<tr>
<td>751</td>
<td>DR-3</td>
<td>1930-02-26</td>
</tr>
<tr>
<td>752</td>
<td>DR-3</td>
<td>None</td>
</tr>
<tr>
<td>837</td>
<td>MSK-4</td>
<td>1932-01-14</td>
</tr>
<tr>
<td>844</td>
<td>DR-1</td>
<td>1932-03-22</td>
</tr>
</tbody>
</table>
<p><strong>Survey</strong>: the actual readings.  The field <code style="color: inherit">quant</code> is short for quantitative and indicates what is being measured.  Values are <code style="color: inherit">rad</code>, <code style="color: inherit">sal</code>, and <code style="color: inherit">temp</code> referring to ‘radiation’, ‘salinity’ and ‘temperature’, respectively.</p>
<table>
<thead>
<tr>
<th>taken</th>
<th>person</th>
<th>quant</th>
<th>reading</th>
</tr>
</thead>
<tbody>
<tr>
<td>619</td>
<td>dyer</td>
<td>rad</td>
<td>9.82</td>
</tr>
<tr>
<td>619</td>
<td>dyer</td>
<td>sal</td>
<td>0.13</td>
</tr>
<tr>
<td>622</td>
<td>dyer</td>
<td>rad</td>
<td>7.8</td>
</tr>
<tr>
<td>622</td>
<td>dyer</td>
<td>sal</td>
<td>0.09</td>
</tr>
<tr>
<td>734</td>
<td>pb</td>
<td>rad</td>
<td>8.41</td>
</tr>
<tr>
<td>734</td>
<td>lake</td>
<td>sal</td>
<td>0.05</td>
</tr>
<tr>
<td>734</td>
<td>pb</td>
<td>temp</td>
<td>-21.5</td>
</tr>
<tr>
<td>735</td>
<td>pb</td>
<td>rad</td>
<td>7.22</td>
</tr>
<tr>
<td>735</td>
<td>None</td>
<td>sal</td>
<td>0.06</td>
</tr>
<tr>
<td>735</td>
<td>None</td>
<td>temp</td>
<td>-26.0</td>
</tr>
<tr>
<td>751</td>
<td>pb</td>
<td>rad</td>
<td>4.35</td>
</tr>
<tr>
<td>751</td>
<td>pb</td>
<td>temp</td>
<td>-18.5</td>
</tr>
<tr>
<td>751</td>
<td>lake</td>
<td>sal</td>
<td>0.1</td>
</tr>
<tr>
<td>752</td>
<td>lake</td>
<td>rad</td>
<td>2.19</td>
</tr>
<tr>
<td>752</td>
<td>lake</td>
<td>sal</td>
<td>0.09</td>
</tr>
<tr>
<td>752</td>
<td>lake</td>
<td>temp</td>
<td>-16.0</td>
</tr>
<tr>
<td>752</td>
<td>roe</td>
<td>sal</td>
<td>41.6</td>
</tr>
<tr>
<td>837</td>
<td>lake</td>
<td>rad</td>
<td>1.46</td>
</tr>
<tr>
<td>837</td>
<td>lake</td>
<td>sal</td>
<td>0.21</td>
</tr>
<tr>
<td>837</td>
<td>roe</td>
<td>sal</td>
<td>22.5</td>
</tr>
<tr>
<td>844</td>
<td>roe</td>
<td>rad</td>
<td>11.25</td>
</tr>
</tbody>
</table>
<p>Notice that three entries — one in the <code style="color: inherit">Visited</code> table,
and two in the <code style="color: inherit">Survey</code> table — don’t contain any actual
data, but instead have a special <code style="color: inherit">None</code> entry:
we’ll return to these missing values.</p>
<p>For now,
let’s write an SQL query that displays scientists’ names.
We do this using the SQL command <code style="color: inherit">SELECT</code>,
giving it the names of the columns we want and the table we want them from.
Our query and its output look like this:</p>


In [7]:
%%sql
SELECT family, personal FROM Person;

Done.


family,personal
Dyer,William
Pabodie,Frank
Lake,Anderson
Roerich,Valentina
Danforth,Frank


<p>The semicolon at the end of the query
tells the database manager that the query is complete and ready to run.
We have written our commands in upper case and the names for the table and columns
in lower case,
but we don’t have to:
as the example below shows,
SQL is case insensitive.</p>


In [8]:
%%sql
SeLeCt FaMiLy, PeRsOnAl FrOm PeRsOn;

Done.


family,personal
Dyer,William
Pabodie,Frank
Lake,Anderson
Roerich,Valentina
Danforth,Frank


<p>You can use SQL’s case insensitivity to your advantage. For instance,
some people choose to write SQL keywords (such as <code style="color: inherit">SELECT</code> and <code style="color: inherit">FROM</code>)
in capital letters and <strong>field</strong> and <strong>table</strong> names in lower
case. This can make it easier to locate parts of an SQL statement. For
instance, you can scan the statement, quickly locate the prominent
<code style="color: inherit">FROM</code> keyword and know the table name follows.  Whatever casing
convention you choose, please be consistent: complex queries are hard
enough to read without the extra cognitive load of random
capitalization.  One convention is to use UPPER CASE for SQL
statements, to distinguish them from tables and column names. This is
the convention that we will use for this lesson.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-is-a-personal-and-family-name-column-a-good-design"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Is a personal and family name column a good design?</div>
<p>If you were tasked with designing a database to store this same data, is storing the name data in
this way the best way to do it? Why or why not?</p>
<p>Can you think of any names that would be difficult to enter in such a schema?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution"><button class="gtn-boxify-button solution" type="button" aria-controls="solution" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>No, it is generally not. There are a lot of <a href="https://shinesolutions.com/2018/01/08/falsehoods-programmers-believe-about-names-with-examples/">falsehoods that programmers believe about names</a>.
The situation is much more complex as you can read in that article, but names vary wildly and
generally placing constraints on how names are entered is only likely to frustrate you or your
users later on when they need to enter data into that database.</p>
<p>In general you should consider using a single text field for the name and allowing users to
specify them as whatever they like (if it is a system with registration), or asking what they
wish to be recorded (if you are doing this sort of data collection).</p>
<p>If you are doing scientific research, you might know that names are generally very poor
identifiers of a single human, and in that case consider recording their
<a href="https://orcid.org/">ORCiD</a> which will help you reference that individual when you are
publishing later.</p>
<p>This is also a good time to consider what data you really <em>need</em> to collect. If you are working
in the EU under GDPR, do you really need their full legal name? Is that necessary? Do you have a
plan for ensuring that data is correct when publishing, if any part of their name has changed
since?</p>
</details>
</blockquote>


In [9]:
%%sql
-- Try solutions here!

Done.


[]

<p>While we are on the topic of SQL’s syntax, one aspect of SQL’s syntax
that can frustrate novices and experts alike is forgetting to finish a
command with <code style="color: inherit">;</code> (semicolon).  When you press enter for a command
without adding the <code style="color: inherit">;</code> to the end, it can look something like this:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT id FROM Person
...&gt;
...&gt;
</code></pre></div></div>
<p>This is SQL’s prompt, where it is waiting for additional commands or
for a <code style="color: inherit">;</code> to let SQL know to finish.  This is easy to fix!  Just type
<code style="color: inherit">;</code> and press enter!</p>
<p>Now, going back to our query,
it’s important to understand that
the rows and columns in a database table aren’t actually stored in any particular order.
They will always be <em>displayed</em> in some order,
but we can control that in various ways.
For example,
we could swap the columns in the output by writing our query as:</p>


In [10]:
%%sql
SELECT personal, family FROM Person;

Done.


personal,family
William,Dyer
Frank,Pabodie
Anderson,Lake
Valentina,Roerich
Frank,Danforth


<p>or even repeat columns:</p>


In [11]:
%%sql
SELECT id, id, id FROM Person;

Done.


id,id_1,id_2
dyer,dyer,dyer
pb,pb,pb
lake,lake,lake
roe,roe,roe
danforth,danforth,danforth


<p>As a shortcut,
we can select all of the columns in a table using <code style="color: inherit">*</code>:</p>


In [12]:
%%sql
SELECT * FROM Person;

Done.


id,personal,family
dyer,William,Dyer
pb,Frank,Pabodie
lake,Anderson,Lake
roe,Valentina,Roerich
danforth,Frank,Danforth


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-selecting-site-names"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Selecting Site Names</div>
<p>Write a query that selects only the <code style="color: inherit">name</code> column from the <code style="color: inherit">Site</code> table.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-1"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-1" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT name FROM Site;
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>name</th>
</tr>
</thead>
<tbody>
<tr>
<td>DR-1</td>
</tr>
<tr>
<td>DR-3</td>
</tr>
<tr>
<td>MSK-4</td>
</tr>
</tbody>
</table>
</details>
</blockquote>


In [13]:
%%sql
-- Try solutions here!
SELECT name FROM Site;

Done.


name
DR-1
DR-3
MSK-4


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-query-style"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Query Style</div>
<p>Many people format queries as:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT personal, family FROM person;
</code></pre></div>  </div>
<p>or as:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select Personal, Family from PERSON;
</code></pre></div>  </div>
<p>What style do you find easiest to read, and why?</p>
</blockquote>


In [14]:
%%sql
-- Try solutions here!

Done.


[]

<h1 id="sorting-and-removing-duplicates">Sorting and Removing Duplicates</h1>
<p>In beginning our examination of the Antarctic data, we want to know:</p>
<ul>
<li>what kind of quantity measurements were taken at each site;</li>
<li>which scientists took measurements on the expedition;</li>
</ul>
<p>To determine which measurements were taken at each site,
we can examine the <code style="color: inherit">Survey</code> table.
Data is often redundant,
so queries often return redundant information.
For example,
if we select the quantities that have been measured
from the <code style="color: inherit">Survey</code> table,
we get this:</p>


In [15]:
%%sql
SELECT quant FROM Survey;

Done.


quant
rad
sal
rad
sal
rad
sal
temp
rad
sal
temp


<p>This result makes it difficult to see all of the different types of
<code style="color: inherit">quant</code> in the Survey table.  We can eliminate the redundant output to
make the result more readable by adding the <code style="color: inherit">DISTINCT</code> keyword to our
query:</p>


In [16]:
%%sql
SELECT DISTINCT quant FROM Survey;

Done.


quant
rad
sal
temp


<p>If we want to determine which visit (stored in the <code style="color: inherit">taken</code> column)
have which <code style="color: inherit">quant</code> measurement,
we can use the <code style="color: inherit">DISTINCT</code> keyword on multiple columns.
If we select more than one column,
distinct <em>sets</em> of values are returned
(in this case <em>pairs</em>, because we are selecting two columns):</p>


In [17]:
%%sql
SELECT DISTINCT taken, quant FROM Survey;

Done.


taken,quant
619,rad
619,sal
622,rad
622,sal
734,rad
734,sal
734,temp
735,rad
735,sal
735,temp


<p>Notice in both cases that duplicates are removed
even if the rows they come from didn’t appear to be adjacent in the database table.</p>
<p>Our next task is to identify the scientists on the expedition by looking at the <code style="color: inherit">Person</code> table.
As we mentioned earlier,
database records are not stored in any particular order.
This means that query results aren’t necessarily sorted,
and even if they are,
we often want to sort them in a different way,
e.g., by their identifier instead of by their personal name.
We can do this in SQL by adding an <code style="color: inherit">ORDER BY</code> clause to our query:</p>


In [18]:
%%sql
SELECT * FROM Person ORDER BY id;

Done.


id,personal,family
danforth,Frank,Danforth
dyer,William,Dyer
lake,Anderson,Lake
pb,Frank,Pabodie
roe,Valentina,Roerich


<table>
<thead>
<tr>
<th>id</th>
<th>personal</th>
<th>family</th>
</tr>
</thead>
<tbody>
<tr>
<td>danfort</td>
<td>Frank</td>
<td>Danforth</td>
</tr>
<tr>
<td>dyer</td>
<td>William</td>
<td>Dyer</td>
</tr>
<tr>
<td>lake</td>
<td>Anderson</td>
<td>Lake</td>
</tr>
<tr>
<td>pb</td>
<td>Frank</td>
<td>Pabodie</td>
</tr>
<tr>
<td>roe</td>
<td>Valentina</td>
<td>Roerich</td>
</tr>
</tbody>
</table>
<p>By default, when we use <code style="color: inherit">ORDER BY</code>,
results are sorted in ascending order of the column we specify
(i.e.,
from least to greatest).</p>
<p>We can sort in the opposite order using <code style="color: inherit">DESC</code> (for “descending”):</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-a-note-on-ordering"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-a-note-on-ordering" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: A note on ordering</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>While it may look that the records are consistent every time we ask for them in this lesson, that is because no one has changed or modified any of the data so far. Remember to use <code style="color: inherit">ORDER BY</code> if you want the rows returned to have any sort of consistent or predictable order.</p>
</blockquote>


In [19]:
%%sql
SELECT * FROM person ORDER BY id DESC;

Done.


id,personal,family
roe,Valentina,Roerich
pb,Frank,Pabodie
lake,Anderson,Lake
dyer,William,Dyer
danforth,Frank,Danforth


<p>(And if we want to make it clear that we’re sorting in ascending order,
we can use <code style="color: inherit">ASC</code> instead of <code style="color: inherit">DESC</code>.)</p>
<p>In order to look at which scientist measured quantities during each visit,
we can look again at the <code style="color: inherit">Survey</code> table.
We can also sort on several fields at once.
For example,
this query sorts results first in ascending order by <code style="color: inherit">taken</code>,
and then in descending order by <code style="color: inherit">person</code>
within each group of equal <code style="color: inherit">taken</code> values:</p>


In [20]:
%%sql
SELECT taken, person, quant FROM Survey ORDER BY taken ASC, person DESC;

Done.


taken,person,quant
619,dyer,rad
619,dyer,sal
622,dyer,rad
622,dyer,sal
734,pb,rad
734,pb,temp
734,lake,sal
735,pb,rad
735,None,sal
735,None,temp


<p>This query gives us a good idea of which scientist was involved in which visit,
and what measurements they performed during the visit.</p>
<p>Looking at the table, it seems like some scientists specialized in
certain kinds of measurements.  We can examine which scientists
performed which measurements by selecting the appropriate columns and
removing duplicates.</p>


In [21]:
%%sql
SELECT DISTINCT quant, person FROM Survey ORDER BY quant ASC;

Done.


quant,person
rad,dyer
rad,pb
rad,lake
rad,roe
sal,dyer
sal,lake
sal,None
sal,roe
temp,pb
temp,None


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-finding-distinct-dates"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Finding Distinct Dates</div>
<p>Write a query that selects distinct dates from the <code style="color: inherit">Visited</code> table.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-2"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-2" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT DISTINCT dated FROM Visited;
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>dated</th>
</tr>
</thead>
<tbody>
<tr>
<td>1927-02-08</td>
</tr>
<tr>
<td>1927-02-10</td>
</tr>
<tr>
<td>1930-01-07</td>
</tr>
<tr>
<td>1930-01-12</td>
</tr>
<tr>
<td>1930-02-26</td>
</tr>
<tr>
<td> </td>
</tr>
<tr>
<td>1932-01-14</td>
</tr>
<tr>
<td>1932-03-22</td>
</tr>
</tbody>
</table>
</details>
</blockquote>


In [22]:
%%sql
-- Try solutions here!
SELECT DISTINCT dated FROM Visited;

Done.


dated
1927-02-08
1927-02-10
1930-01-07
1930-01-12
1930-02-26
None
1932-01-14
1932-03-22


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-displaying-full-names"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Displaying Full Names</div>
<p>Write a query that displays the full names of the scientists in the <code style="color: inherit">Person</code> table,
ordered by family name.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-3"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-3" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT personal, family FROM Person ORDER BY family ASC;
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>personal</th>
<th>family</th>
</tr>
</thead>
<tbody>
<tr>
<td>Frank</td>
<td>Danforth</td>
</tr>
<tr>
<td>William</td>
<td>Dyer</td>
</tr>
<tr>
<td>Anderson</td>
<td>Lake</td>
</tr>
<tr>
<td>Frank</td>
<td>Pabodie</td>
</tr>
<tr>
<td>Valentina</td>
<td>Roerich</td>
</tr>
</tbody>
</table>
</details>
</blockquote>


In [23]:
%%sql
-- Try solutions here!
SELECT personal, family FROM Person ORDER BY family

Done.


personal,family
Frank,Danforth
William,Dyer
Anderson,Lake
Frank,Pabodie
Valentina,Roerich


<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-is-sorting-names-useful"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-is-sorting-names-useful" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: Is sorting names useful?</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>If you are someone with a name which falls at the end of the alphabet, you’ve likely been
penalised for this your entire life. Alphabetically sorting names should always be looked at
critically and through a lens to whether you are fairly reflecting everyone’s contributions,
rather than just the default sort order.</p>
<p>There are many options, either by some metric of contribution that everyone could agree on, or
better, consider random sorting, like the GTN uses with our <a href="{% link hall-of-fame.md %}">Hall of Fame</a>
page where we intentionally order randomly to tell contributors that no one persons
contributions matter more than anothers.</p>
<blockquote class="quote">
<p>The evidence provided in a variety of studies leaves no doubt that an
alphabetical author ordering norm disadvantages researchers with
last names toward the end of the alphabet. There is furthermore con-
vincing evidence that researchers are aware of this and that they
react strategically to such alphabetical discrimination, for example
with their choices of who to collaborate with. See {% cite Weber_2018 %} for more.</p>
</blockquote>
</blockquote>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-name-collation"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-name-collation" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: Name collation</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>When you are sorting things in SQL, you need to be aware of something called collation which can
affect your results if you have values that are not the letters A-Z. Collating is the process of
sorting values, and this affects many human languages when storing data in a database.</p>
<p>Here is a Dutch example. In the old days their alphabet contained a <code style="color: inherit">ÿ</code> which was later replaced
with <code style="color: inherit">ĳ</code>, a digraph of two characters squished together. This is commonly rendered as <code style="color: inherit">ij</code>
however, two separate characters, due to the internet and widespread use of keyboards featuring
mainly ascii characters. However, it is still the 25th letter of their alphabet.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">sqlite&gt; create table nl(value text);
sqlite&gt; insert into nl values ('appel'), ('beer'), ('index'), ('ijs'), ('jammer'), ('winkel'), ('zon');
sqlite&gt; select * from nl order by value;
appel
beer
index
ijs
jammer
winkel
zon
</code></pre></div>  </div>
<p>Find a dutch friend and ask them if this is the correct order for this list. Unfortunately it
isn’t. Even though it is <code style="color: inherit">ij</code> as two separate characters, it should be sorted as if it was <code style="color: inherit">ĳ</code> or
<code style="color: inherit">ÿ</code>, before <code style="color: inherit">z</code>. Like so: appel, beer, index, jammer, winkel, ijs, zon</p>
<p>While there is not much you can do about it now (you’re just beginning!) it is something you
should be aware of. When you later need to know about this, you will find the term ‘collation’
useful, and you’ll find the procedure is different for every database engine.</p>
</blockquote>
<h1 id="filtering">Filtering</h1>
<p>One of the most powerful features of a database is
the ability to filter data,
i.e.,
to select only those records that match certain criteria.
For example,
suppose we want to see when a particular site was visited.
We can select these records from the <code style="color: inherit">Visited</code> table
by using a <code style="color: inherit">WHERE</code> clause in our query:</p>


In [24]:
%%sql
SELECT * FROM Visited WHERE site = 'DR-1';

Done.


id,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10
844,DR-1,1932-03-22


<p>The database manager executes this query in two stages.
First,
it checks at each row in the <code style="color: inherit">Visited</code> table
to see which ones satisfy the <code style="color: inherit">WHERE</code>.
It then uses the column names following the <code style="color: inherit">SELECT</code> keyword
to determine which columns to display.</p>
<p>This processing order means that
we can filter records using <code style="color: inherit">WHERE</code>
based on values in columns that aren’t then displayed:</p>


In [25]:
%%sql
SELECT id FROM Visited WHERE site = 'DR-1';

Done.


id
619
622
844


<p>![SQL Filtering in Action]`(../../images/carpentries-sql/sql-filter.svg)</p>
<p>We can use many other Boolean operators to filter our data.
For example,
we can ask for all information from the DR-1 site collected before 1930:</p>


In [26]:
%%sql
SELECT * FROM Visited WHERE site = 'DR-1' AND dated < '1930-01-01';

Done.


id,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10


<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-date-types"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-date-types" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: Date Types</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>Most database managers have a special data type for dates.
In fact, many have two:
one for dates,
such as “May 31, 1971”,
and one for durations,
such as “31 days”.
SQLite doesn’t:
instead,
it stores dates as either text
(in the ISO-8601 standard format “YYYY-MM-DD HH:MM:SS.SSSS”),
real numbers
(<a href="https://en.wikipedia.org/wiki/Julian_day">Julian days</a>, the number of days since November 24, 4714 BCE),
or integers
(<a href="https://en.wikipedia.org/wiki/Unix_time">Unix time</a>, the number of seconds since midnight, January 1, 1970).
If this sounds complicated,
it is,
but not nearly as complicated as figuring out
<a href="https://en.wikipedia.org/wiki/Swedish_calendar">historical dates in Sweden</a>.</p>
</blockquote>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-30-1930-or-2030"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-30-1930-or-2030" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: '30: 1930 or 2030?</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>Storing the year as the last two digits causes problems in databases, and is part of what caused
<a href="https://en.wikipedia.org/wiki/Year_2000_problem">Y2K</a>. Be sure to use the databases’ built in
format for storing dates, if it is available as that will generally avoid any major issues.</p>
<p>Similarly there is a <a href="https://en.wikipedia.org/wiki/Year_2000_problem#Year_2038_problem">“Year 2038 problem”</a>,
as the timestamps mentioned above that count seconds since Jan 1, 1970 were running out of space
on 32-bit machines. Many systems have since migrated to work around this with 64-bit timestamps.</p>
</blockquote>
<p>If we want to find out what measurements were taken by either Lake or Roerich,
we can combine the tests on their names using <code style="color: inherit">OR</code>:</p>


In [27]:
%%sql
SELECT * FROM Survey WHERE person = 'lake' OR person = 'roe';

Done.


taken,person,quant,reading
734,lake,sal,0.05
751,lake,sal,0.1
752,lake,rad,2.19
752,lake,sal,0.09
752,lake,temp,-16.0
752,roe,sal,41.6
837,lake,rad,1.46
837,lake,sal,0.21
837,roe,sal,22.5
844,roe,rad,11.25


<p>Alternatively,
we can use <code style="color: inherit">IN</code> to see if a value is in a specific set:</p>


In [28]:
%%sql
SELECT * FROM Survey WHERE person IN ('lake', 'roe');

Done.


taken,person,quant,reading
734,lake,sal,0.05
751,lake,sal,0.1
752,lake,rad,2.19
752,lake,sal,0.09
752,lake,temp,-16.0
752,roe,sal,41.6
837,lake,rad,1.46
837,lake,sal,0.21
837,roe,sal,22.5
844,roe,rad,11.25


<p>We can combine <code style="color: inherit">AND</code> with <code style="color: inherit">OR</code>,
but we need to be careful about which operator is executed first.
If we <em>don’t</em> use parentheses,
we get this:</p>


In [29]:
%%sql
SELECT * FROM Survey WHERE quant = 'sal' AND person = 'lake' OR person = 'roe';

Done.


taken,person,quant,reading
734,lake,sal,0.05
751,lake,sal,0.1
752,lake,sal,0.09
752,roe,sal,41.6
837,lake,sal,0.21
837,roe,sal,22.5
844,roe,rad,11.25


<p>which is salinity measurements by Lake,
and <em>any</em> measurement by Roerich.
We probably want this instead:</p>


In [30]:
%%sql
SELECT * FROM Survey WHERE quant = 'sal' AND (person = 'lake' OR person = 'roe');

Done.


taken,person,quant,reading
734,lake,sal,0.05
751,lake,sal,0.1
752,lake,sal,0.09
752,roe,sal,41.6
837,lake,sal,0.21
837,roe,sal,22.5


<p>We can also filter by partial matches.  For example, if we want to
know something just about the site names beginning with “DR” we can
use the <code style="color: inherit">LIKE</code> keyword.  The percent symbol acts as a
wildcard, matching any characters in that
place.  It can be used at the beginning, middle, or end of the string
<a href="https://www.w3schools.com/sql/sql_wildcards.asp">See this page on wildcards</a> for more information:</p>


In [31]:
%%sql
SELECT * FROM Visited WHERE site LIKE 'DR%';

Done.


id,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10
734,DR-3,1930-01-07
735,DR-3,1930-01-12
751,DR-3,1930-02-26
752,DR-3,None
844,DR-1,1932-03-22


<p>Finally,
we can use <code style="color: inherit">DISTINCT</code> with <code style="color: inherit">WHERE</code>
to give a second level of filtering:</p>


In [32]:
%%sql
SELECT DISTINCT person, quant FROM Survey WHERE person = 'lake' OR person = 'roe';

Done.


person,quant
lake,sal
lake,rad
lake,temp
roe,sal
roe,rad


<p>But remember:
<code style="color: inherit">DISTINCT</code> is applied to the values displayed in the chosen columns,
not to the entire rows as they are being processed.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-growing-queries"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-growing-queries" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: Growing Queries</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>What we have just done is how most people “grow” their <abbr title="Structured Query Language">SQL</abbr> queries.
We started with something simple that did part of what we wanted,
then added more clauses one by one,
testing their effects as we went.
This is a good strategy — in fact,
for complex queries it’s often the <em>only</em> strategy — but
it depends on quick turnaround,
and on us recognizing the right answer when we get it.</p>
<p>The best way to achieve quick turnaround is often
to put a subset of data in a temporary database
and run our queries against that,
or to fill a small database with synthesized records.
For example,
instead of trying our queries against an actual database of 20 million Australians,
we could run it against a sample of ten thousand,
or write a small program to generate ten thousand random (but plausible) records
and use that.</p>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-fix-this-query"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Fix This Query</div>
<p>Suppose we want to select all sites that lie within 48 degrees of the equator.
Our first query is:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * FROM Site WHERE (lat &gt; -48) OR (lat &lt; 48);
</code></pre></div>  </div>
<p>Explain why this is wrong,
and rewrite the query so that it is correct.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-4"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-4" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>Because we used <code style="color: inherit">OR</code>, a site on the South Pole for example will still meet
the second criteria and thus be included. Instead, we want to restrict this
to sites that meet <em>both</em> criteria:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * FROM Site WHERE (lat &gt; -48) AND (lat &lt; 48);
</code></pre></div>    </div>
</details>
</blockquote>


In [33]:
%%sql
-- Try solutions here!
SELECT * FROM Site WHERE (lat > -48) AND (lat < 48);

Done.


name,lat,long
DR-3,-47.15,-126.72


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-finding-outliers"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Finding Outliers</div>
<p>Normalized salinity readings are supposed to be between 0.0 and 1.0.
Write a query that selects all records from <code style="color: inherit">Survey</code>
with salinity values outside this range.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-5"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-5" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * FROM Survey WHERE quant = 'sal' AND ((reading &gt; 1.0) OR (reading &lt; 0.0));
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>taken</th>
<th>person</th>
<th>quant</th>
<th>reading</th>
</tr>
</thead>
<tbody>
<tr>
<td>752</td>
<td>roe</td>
<td>sal</td>
<td>41.6</td>
</tr>
<tr>
<td>837</td>
<td>roe</td>
<td>sal</td>
<td>22.5</td>
</tr>
</tbody>
</table>
</details>
</blockquote>


In [34]:
%%sql
-- Try solutions here!
SELECT * FROM Survey WHERE quant = 'sal' AND ((reading > 1.0) OR (reading < 0.0))

Done.


taken,person,quant,reading
752,roe,sal,41.6
837,roe,sal,22.5


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-matching-patterns"><i class="far fa-question-circle" aria-hidden="true" ></i> Question:  Matching Patterns</div>
<p>Which of these expressions are true?</p>
<ol>
<li><code style="color: inherit">'a' LIKE 'a'</code></li>
<li><code style="color: inherit">'a' LIKE '%a'</code></li>
<li><code style="color: inherit">'beta' LIKE '%a'</code></li>
<li><code style="color: inherit">'alpha' LIKE 'a%%'</code></li>
<li><code style="color: inherit">'alpha' LIKE 'a%p%'</code></li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-6"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-6" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>True because these are the same character.</li>
<li>True because the wildcard can match <em>zero</em> or more characters.</li>
<li>True because the <code style="color: inherit">%</code> matches <code style="color: inherit">bet</code> and the <code style="color: inherit">a</code> matches the <code style="color: inherit">a</code>.</li>
<li>True because the first wildcard matches <code style="color: inherit">lpha</code> and the second wildcard matches zero characters (or vice versa).</li>
<li>True because the first wildcard matches <code style="color: inherit">l</code> and the second wildcard matches <code style="color: inherit">ha</code>.</li>
</ol>
</details>
</blockquote>


<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-case-insensitive-matching"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-case-insensitive-matching" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: Case-insensitive matching</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>But what about if you don’t care about if it’s <code style="color: inherit">ALPHA</code> or <code style="color: inherit">alpha</code> in the database, and you are
using a language that has a notion of case (unlike e.g. Chinese, Japenese)?</p>
<p>Then you can use the <code style="color: inherit">ILIKE</code> operator for ‘case Insensitive LIKE’.
for example the following are true:</p>
<ul>
<li><code style="color: inherit">'a' ILIKE 'A'</code></li>
<li><code style="color: inherit">'AlPhA' ILIKE '%lpha'</code></li>
</ul>
</blockquote>
<h1 id="calculating-new-values">Calculating New Values</h1>
<p>After carefully re-reading the expedition logs,
we realize that the radiation measurements they report
may need to be corrected upward by 5%.
Rather than modifying the stored data,
we can do this calculation on the fly
as part of our query:</p>


In [35]:
%%sql
SELECT 1.05 * reading FROM Survey WHERE quant = 'rad';

Done.


1.05 * reading
10.311
8.19
8.8305
7.581
4.5675
2.2995
1.533
11.8125


<p>When we run the query,
the expression <code style="color: inherit">1.05 * reading</code> is evaluated for each row.
Expressions can use any of the fields,
all of usual arithmetic operators,
and a variety of common functions.
(Exactly which ones depends on which database manager is being used.)
For example,
we can convert temperature readings from Fahrenheit to Celsius
and round to two decimal places:</p>


In [36]:
%%sql
SELECT taken, round(5 * (reading - 32) / 9, 2) FROM Survey WHERE quant = 'temp';

Done.


taken,"round(5 * (reading - 32) / 9, 2)"
734,-29.72
735,-32.22
751,-28.06
752,-26.67


<p>As you can see from this example, though, the string describing our
new field (generated from the equation) can become quite unwieldy. <abbr title="Structured Query Language">SQL</abbr>
allows us to rename our fields, any field for that matter, whether it
was calculated or one of the existing fields in our database, for
succinctness and clarity. For example, we could write the previous
query as:</p>


In [37]:
%%sql
SELECT taken, round(5 * (reading - 32) / 9, 2) as Celsius FROM Survey WHERE quant = 'temp';

Done.


taken,Celsius
734,-29.72
735,-32.22
751,-28.06
752,-26.67


<p>We can also combine values from different fields,
for example by using the string concatenation operator <code style="color: inherit">||</code>:</p>


In [38]:
%%sql
SELECT personal || ' ' || family FROM Person;

Done.


personal || ' ' || family
William Dyer
Frank Pabodie
Anderson Lake
Valentina Roerich
Frank Danforth


<p>But of course that can also be solved by simply having a single name field which avoids other
issues.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-fixing-salinity-readings"><i class="far fa-question-circle" aria-hidden="true" ></i> Question:  Fixing Salinity Readings</div>
<p>After further reading,
we realize that Valentina Roerich
was reporting salinity as percentages.
Write a query that returns all of her salinity measurements
from the <code style="color: inherit">Survey</code> table
with the values divided by 100.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-7"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-7" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT taken, reading / 100 FROM Survey WHERE person = 'roe' AND quant = 'sal';
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>taken</th>
<th>reading / 100</th>
</tr>
</thead>
<tbody>
<tr>
<td>752</td>
<td>0.416</td>
</tr>
<tr>
<td>837</td>
<td>0.225</td>
</tr>
</tbody>
</table>
</details>
</blockquote>


In [39]:
%%sql
-- Try solutions here!
SELECT taken, reading / 100 FROM Survey WHERE person = 'roe' and quant = 'sal'

Done.


taken,reading / 100
752,0.41600000000000004
837,0.225


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-unions"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Unions</div>
<p>The <code style="color: inherit">UNION</code> operator combines the results of two queries:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * FROM Person WHERE id = 'dyer' UNION SELECT * FROM Person WHERE id = 'roe';
</code></pre></div>  </div>
<table>
<thead>
<tr>
<th>id</th>
<th>personal</th>
<th>family</th>
</tr>
</thead>
<tbody>
<tr>
<td>dyer</td>
<td>William</td>
<td>Dyer</td>
</tr>
<tr>
<td>roe</td>
<td>Valentina</td>
<td>Roerich</td>
</tr>
</tbody>
</table>
<p>The <code style="color: inherit">UNION ALL</code> command is equivalent to the <code style="color: inherit">UNION</code> operator,
except that <code style="color: inherit">UNION ALL</code> will select all values.
The difference is that <code style="color: inherit">UNION ALL</code> will not eliminate duplicate rows.
Instead, <code style="color: inherit">UNION ALL</code> pulls all rows from the query
specifics and combines them into a table.
The <code style="color: inherit">UNION</code> command does a <code style="color: inherit">SELECT DISTINCT</code> on the results set.
If all the records to be returned are unique from your union,
use <code style="color: inherit">UNION ALL</code> instead, it gives faster results since it skips the <code style="color: inherit">DISTINCT</code> step.
For this section, we shall use UNION.</p>
<p>Use <code style="color: inherit">UNION</code> to create a consolidated list of salinity measurements
in which Valentina Roerich’s, and only Valentina’s,
have been corrected as described in the previous challenge.
The output should be something like:</p>
<table>
<thead>
<tr>
<th>taken</th>
<th>reading</th>
</tr>
</thead>
<tbody>
<tr>
<td>619</td>
<td>0.13</td>
</tr>
<tr>
<td>622</td>
<td>0.09</td>
</tr>
<tr>
<td>734</td>
<td>0.05</td>
</tr>
<tr>
<td>751</td>
<td>0.1</td>
</tr>
<tr>
<td>752</td>
<td>0.09</td>
</tr>
<tr>
<td>752</td>
<td>0.416</td>
</tr>
<tr>
<td>837</td>
<td>0.21</td>
</tr>
<tr>
<td>837</td>
<td>0.225</td>
</tr>
</tbody>
</table>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-8"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-8" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT taken, reading FROM Survey WHERE person != 'roe' AND quant = 'sal' UNION SELECT taken, reading / 100 FROM Survey WHERE person = 'roe' AND quant = 'sal' ORDER BY taken ASC;
</code></pre></div>    </div>
</details>
</blockquote>


In [40]:
%%sql
-- Try solutions here!

Done.


[]

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-selecting-major-site-identifiers"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Selecting Major Site Identifiers</div>
<p>The site identifiers in the <code style="color: inherit">Visited</code> table have two parts
separated by a ‘-‘:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT DISTINCT site FROM Visited;
</code></pre></div>  </div>
<table>
<thead>
<tr>
<th>site</th>
</tr>
</thead>
<tbody>
<tr>
<td>DR-1</td>
</tr>
<tr>
<td>DR-3</td>
</tr>
<tr>
<td>MSK-4</td>
</tr>
</tbody>
</table>
<p>Some major site identifiers (i.e. the letter codes) are two letters long and some are three.
The “in string” function <code style="color: inherit">instr(X, Y)</code>
returns the 1-based index of the first occurrence of string Y in string X,
or 0 if Y does not exist in X.
The substring function <code style="color: inherit">substr(X, I, [L])</code>
returns the substring of X starting at index I, with an optional length L.
Use these two functions to produce a list of unique major site identifiers.
(For this data,
the list should contain only “DR” and “MSK”).</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-9"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-9" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT DISTINCT substr(site, 1, instr(site, '-') - 1) AS MajorSite FROM Visited;
</code></pre></div>    </div>
</details>
</blockquote>


In [41]:
%%sql
-- Try solutions here!
SELECT taken, reading FROM Survey WHERE person != 'roe' AND quant = 'sal' UNION SELECT taken, reading / 100 FROM Survey WHERE person = 'roe' AND quant = 'sal' ORDER BY taken ASC;

Done.


taken,reading
619,0.13
622,0.09
734,0.05
751,0.1
752,0.09
752,0.41600000000000004
837,0.21
837,0.225


<h1 id="missing-data">Missing Data</h1>
<p>Real-world data is never complete — there are always holes.
Databases represent these holes using a special value called <code style="color: inherit">null</code>.
<code style="color: inherit">null</code> is not zero, <code style="color: inherit">False</code>, or the empty string;
it is a one-of-a-kind value that means “nothing here”.
Dealing with <code style="color: inherit">null</code> requires a few special tricks
and some careful thinking.</p>
<p>By default, the Python SQL interface does not display NULL values in its output, instead it shows <code style="color: inherit">None</code>.</p>
<p>To start,
let’s have a look at the <code style="color: inherit">Visited</code> table.
There are eight records,
but #752 doesn’t have a date — or rather,
its date is null:</p>


In [42]:
%%sql
SELECT * FROM Visited;

Done.


id,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10
734,DR-3,1930-01-07
735,DR-3,1930-01-12
751,DR-3,1930-02-26
752,DR-3,None
837,MSK-4,1932-01-14
844,DR-1,1932-03-22


<p>Null doesn’t behave like other values.
If we select the records that come before 1930:</p>


In [43]:
%%sql
SELECT * FROM Visited WHERE dated < '1930-01-01';

Done.


id,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10


<p>we get two results,
and if we select the ones that come during or after 1930:</p>


In [44]:
%%sql
SELECT * FROM Visited WHERE dated >= '1930-01-01';

Done.


id,site,dated
734,DR-3,1930-01-07
735,DR-3,1930-01-12
751,DR-3,1930-02-26
837,MSK-4,1932-01-14
844,DR-1,1932-03-22


<p>we get five,
but record #752 isn’t in either set of results.
The reason is that
<code class="language-plaintext highlighter-rouge">null&lt;'1930-01-01'</code>
is neither true nor false:
null means, “We don’t know,”
and if we don’t know the value on the left side of a comparison,
we don’t know whether the comparison is true or false.
Since databases represent “don’t know” as null,
the value of <code style="color: inherit">null&lt;'1930-01-01'</code>
is actually <code style="color: inherit">null</code>.
<code style="color: inherit">null&gt;='1930-01-01'</code> is also null
because we can’t answer to that question either.
And since the only records kept by a <code style="color: inherit">WHERE</code>
are those for which the test is true,
record #752 isn’t included in either set of results.</p>
<p>Comparisons aren’t the only operations that behave this way with nulls.
<code style="color: inherit">1+null</code> is <code style="color: inherit">null</code>,
<code style="color: inherit">5*null</code> is <code style="color: inherit">null</code>,
<code style="color: inherit">log(null)</code> is <code style="color: inherit">null</code>,
and so on.
In particular,
comparing things to null with = and != produces null:</p>


In [45]:
%%sql
SELECT * FROM Visited WHERE dated = NULL;

Done.


id,site,dated


<p>produces no output, and neither does:</p>


In [46]:
%%sql
SELECT * FROM Visited WHERE dated != NULL;

Done.


id,site,dated


<p>To check whether a value is <code style="color: inherit">null</code> or not,
we must use a special test <code style="color: inherit">IS NULL</code>:</p>


In [47]:
%%sql
SELECT * FROM Visited WHERE dated IS NULL;

Done.


id,site,dated
752,DR-3,None


<p>or its inverse <code style="color: inherit">IS NOT NULL</code>:</p>


In [48]:
%%sql
SELECT * FROM Visited WHERE dated IS NOT NULL;

Done.


id,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10
734,DR-3,1930-01-07
735,DR-3,1930-01-12
751,DR-3,1930-02-26
837,MSK-4,1932-01-14
844,DR-1,1932-03-22


<p>Null values can cause headaches wherever they appear.
For example,
suppose we want to find all the salinity measurements
that weren’t taken by Lake.
It’s natural to write the query like this:</p>


In [49]:
%%sql
SELECT * FROM Survey WHERE quant = 'sal' AND person != 'lake';

Done.


taken,person,quant,reading
619,dyer,sal,0.13
622,dyer,sal,0.09
752,roe,sal,41.6
837,roe,sal,22.5


<p>but this query filters omits the records
where we don’t know who took the measurement.
Once again,
the reason is that when <code style="color: inherit">person</code> is <code style="color: inherit">null</code>,
the <code style="color: inherit">!=</code> comparison produces <code style="color: inherit">null</code>,
so the record isn’t kept in our results.
If we want to keep these records
we need to add an explicit check:</p>


In [50]:
%%sql
SELECT * FROM Survey WHERE quant = 'sal' AND (person != 'lake' OR person IS NULL);

Done.


taken,person,quant,reading
619,dyer,sal,0.13
622,dyer,sal,0.09
735,None,sal,0.06
752,roe,sal,41.6
837,roe,sal,22.5


<p>We still have to decide whether this is the right thing to do or not.
If we want to be absolutely sure that
we aren’t including any measurements by Lake in our results,
we need to exclude all the records for which we don’t know who did the work.</p>
<p>In contrast to arithmetic or Boolean operators, aggregation functions
that combine multiple values, such as <code style="color: inherit">min</code>, <code style="color: inherit">max</code> or <code style="color: inherit">avg</code>, <em>ignore</em>
<code style="color: inherit">null</code> values. In the majority of cases, this is a desirable output:
for example, unknown values are thus not affecting our data when we
are averaging it. Aggregation functions will be addressed in more
detail in <a href="#">the next section</a>.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-sorting-by-known-date"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Sorting by Known Date</div>
<p>Write a query that sorts the records in <code style="color: inherit">Visited</code> by date,
omitting entries for which the date is not known
(i.e., is null).</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-10"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-10" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * FROM Visited WHERE dated IS NOT NULL ORDER BY dated ASC;
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>id</th>
<th>site</th>
<th>dated</th>
</tr>
</thead>
<tbody>
<tr>
<td>619</td>
<td>DR-1</td>
<td>1927-02-08</td>
</tr>
<tr>
<td>622</td>
<td>DR-1</td>
<td>1927-02-10</td>
</tr>
<tr>
<td>734</td>
<td>DR-3</td>
<td>1930-01-07</td>
</tr>
<tr>
<td>735</td>
<td>DR-3</td>
<td>1930-01-12</td>
</tr>
<tr>
<td>751</td>
<td>DR-3</td>
<td>1930-02-26</td>
</tr>
<tr>
<td>837</td>
<td>MSK-4</td>
<td>1932-01-14</td>
</tr>
<tr>
<td>844</td>
<td>DR-1</td>
<td>1932-03-22</td>
</tr>
</tbody>
</table>
</details>
</blockquote>


In [51]:
%%sql
-- Try solutions here!
SELECT * FROM Visited WHERE dated IS NOT NULL ORDER BY dated ASC;

Done.


id,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10
734,DR-3,1930-01-07
735,DR-3,1930-01-12
751,DR-3,1930-02-26
837,MSK-4,1932-01-14
844,DR-1,1932-03-22


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-null-in-a-set"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: NULL in a Set</div>
<p>What do you expect the following query to produce?</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * FROM Visited WHERE dated IN ('1927-02-08', NULL);
</code></pre></div>  </div>
<p>What does it actually produce?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-11"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-11" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>You might expect the above query to return rows where dated is either ‘1927-02-08’ or NULL.
Instead it only returns rows where dated is ‘1927-02-08’, the same as you would get from this
simpler query:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * FROM Visited WHERE dated IN ('1927-02-08');
</code></pre></div>    </div>
<p>The reason is that the <code style="color: inherit">IN</code> operator works with a set of <em>values</em>, but NULL is by definition
not a value and is therefore simply ignored.</p>
<p>If we wanted to actually include NULL, we would have to rewrite the query to use the IS NULL condition:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * FROM Visited WHERE dated = '1927-02-08' OR dated IS NULL;
</code></pre></div>    </div>
</details>
</blockquote>


In [52]:
%%sql
-- Try solutions here!
SELECT * FROM Visited WHERE dated = '1927-02-08' OR dated IS NULL;

Done.


id,site,dated
619,DR-1,1927-02-08
752,DR-3,None


<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-pros-and-cons-of-sentinels"><i class="far fa-question-circle" aria-hidden="true" ></i> Question: Pros and Cons of Sentinels</div>
<p>Some database designers prefer to use
a sentinel value
to mark missing data rather than <code style="color: inherit">null</code>.
For example,
they will use the date “0000-00-00” to mark a missing date,
or -1.0 to mark a missing salinity or radiation reading
(since actual readings cannot be negative).
What does this simplify?
What burdens or risks does it introduce?</p>
</blockquote>


Answer:

Using sentinel values like "0000-00-00" or -1.0 to represent missing data instead of NULL can simplify comparisons and logic in some systems, as these values behave like regular data and avoid SQL's special handling of NULL. However, this approach introduces significant risks: sentinel values can be mistaken for valid data, require additional logic to filter or interpret, and reduce data integrity by violating type expectations (e.g., a date column containing an invalid date). They also hinder compatibility with modern tools and systems that natively support NULL. Overall, while sentinel values may offer short-term convenience, they increase the risk of misinterpretation and data quality issues in the long run.

# Key Points

- A relational database stores information in tables, each of which has a fixed set of columns and a variable number of records.
- A database manager is a program that manipulates information stored in a database.
- We write queries in a specialized language called SQL to extract information from databases.
- Use SELECT... FROM... to get values from a database table.
- SQL is case-insensitive (but data is case-sensitive).
- The records in a database table are not intrinsically ordered: if we want to display them in some order, we must specify that explicitly with ORDER BY.
- The values in a database are not guaranteed to be unique: if we want to eliminate duplicates, we must specify that explicitly as well using DISTINCT.
- Use WHERE to specify conditions that records must meet in order to be included in a query's results.
- Use AND, OR, and NOT to combine tests.
- Filtering is done on whole records, so conditions can use fields that are not actually displayed.
- Write queries incrementally.
- Queries can do the usual arithmetic operations on values.
- Use UNION to combine the results of two or more queries.
- Databases use a special value called NULL to represent missing information.
- Almost all operations on NULL produce NULL.
- Queries can test for NULLs using IS NULL and IS NOT NULL.

# Congratulations on successfully completing this tutorial!

